# Home task 4

1. Download 50-100 pages from WikiPedia that have cross-links (probably pages of the same category).

2. Parse these files and create the DataFrame with pages cross-links:

* column `page_1_name`: parent page name (page that contains a link)
* column `page_1_url`: parent page URL (page that contains a link)
* column `page_2_name`: child page name (page that is referred by parent page)
* column `page_2_url`: child page URL (page that is referred by parent page)

3. Create the DataFrame with `page_url` and `page_id` (1—...) columns (second DataFrame).

4. Use it to create a square-shaped DataFrame (third DataFrame) with the same page ids in the index and columns (symmetric index and column names). The third DataFrame should contain zeros and ones (0 and 1). Value `1` at `.loc[1, 2]` position in the DataFrame means that the page with id `1` is referring to page with id `2`.

In [2]:
# wget рекурсивная загрузка – ссылки должны быть домен википедия
# robots = off который запрещает загрузку

# https://beautiful-soup-4.readthedocs.io/en/latest/

In [1]:
import pandas as pd
import itertools
import timeit
import re

In [2]:
!pip install requests
!pip install beautifulsoup4

In [3]:
import requests as req
from bs4 import BeautifulSoup

In [4]:
# let's work with wiki-page 'История России'
result = req.get('https://ru.wikipedia.org/wiki/%D0%98%D1%81%D1%82%D0%BE%D1%80%D0%B8%D1%8F_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
parser = BeautifulSoup(result.content)

In [5]:
a = parser.find('div', {'class':"mw-body-content mw-content-ltr"})
href = a.select('p > a')

In [6]:
# filtering links that starts with '/wiki'

links = list(map(lambda x: x.get('href'), href))
r = re.compile("/wiki")
filtered_links = list(filter(r.match, links)) 

filtered_href = []
for i in href:
    if i.get('href') in filtered_links:
        filtered_href.append(i)
    else:
        continue

len(filtered_href)

1138

In [7]:
# add https to filtered links  
links = ['https://ru.wikipedia.org' + link.get('href') for link in filtered_href]    

In [8]:
# extract names of urls
subjects_names = list(map(lambda x: x.get('title'), filtered_href))
len(subjects_names)

1138

In [9]:
# decrease the number of parent pages
dict_df = {'page_1_name':subjects_names, 'page_1_url':links}
df_wiki = pd.DataFrame(dict_df)
df_wiki1 = df_wiki[:100]

In [10]:
len(df_wiki1['page_1_name'].unique()) # there are several dublicates

91

In [11]:
# let's delete dublicates
df_wiki1.drop_duplicates(inplace = True, ignore_index=True)
df_wiki1

/Users/macbook/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,page_1_name,page_1_url
0,Восточные славяне,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...
1,Восточно-Европейская равнина,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...
2,VI,https://ru.wikipedia.org/wiki/VI
3,Русские,https://ru.wikipedia.org/wiki/%D0%A0%D1%83%D1%...
4,Украинцы,https://ru.wikipedia.org/wiki/%D0%A3%D0%BA%D1%...
...,...,...
86,Октябрьский переворот,https://ru.wikipedia.org/wiki/%D0%9E%D0%BA%D1%...
87,Установление советской власти в России,https://ru.wikipedia.org/wiki/%D0%A3%D1%81%D1%...
88,Российская Социалистическая Федеративная Совет...,https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D1%...
89,Распад Российской империи,https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%...


In [12]:
child_pages = []
page_1_name = []
page_1_url = [] 

def get_links(df):
    for i in range(len(df)):
        res = req.get(df['page_1_url'][i])
        pars = BeautifulSoup(res.content)
        links = pars.find('div', {'class':"mw-body-content mw-content-ltr"}).select('p > a')
        links_filtered = []
        for j in links:
            if j.get('href').startswith('/wiki') == True:
                links_filtered.append(j)
            else:
                continue
        child_pages.extend(links_filtered[:100])
        
        page_1_name.extend(itertools.repeat(df['page_1_name'][i], len(links_filtered[:100])))
        page_1_url.extend(itertools.repeat(df['page_1_url'][i], len(links_filtered[:100])))
    return f'Number of child pages: {len(child_pages)}'

get_links(df_wiki1)

AttributeError: 'NoneType' object has no attribute 'startswith'

In [201]:
%%timeit
get_links(df_wiki1)

27.7 s ± 81.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
page_2_name = list(map(lambda x: x.get('title'), child_pages))
page_2_url = list(map(lambda x: 'https://ru.wikipedia.org' + x.get('href'), child_pages))
                   
final_wiki = pd.DataFrame({'page_1_name': page_1_name, 
                           'page_1_url': page_1_url, 
                           'page_2_name': page_2_name,
                           'page_2_url': page_2_url})
final_wiki

,page_1_name,page_1_url,page_2_name,page_2_url
0,Восточные славяне,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...,Славяне,https://ru.wikipedia.org/wiki/%D0%A1%D0%BB%D0%...
1,Восточные славяне,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...,Восточнославянские языки,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...
2,Восточные славяне,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...,Племенные союзы восточных славян,https://ru.wikipedia.org/wiki/%D0%9F%D0%BB%D0%...
3,Восточные славяне,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...,Древнерусская народность,https://ru.wikipedia.org/wiki/%D0%94%D1%80%D0%...
4,Восточные славяне,https://ru.wikipedia.org/wiki/%D0%92%D0%BE%D1%...,Средние века,https://ru.wikipedia.org/wiki/%D0%A1%D1%80%D0%...
...,...,...,...,...
8145,Российская Социалистическая Федеративная Совет...,https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D1%...,Верхнеуральск,https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D1%...
8146,Российская Социалистическая Федеративная Совет...,https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D1%...,Красная армия,https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%...
8147,Российская Социалистическая Федеративная Совет...,https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D1%...,Краснинский (Челябинская область),https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%...
8148,Российская Социалистическая Федеративная Совет...,https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D1%...,Тургайская степь,https://ru.wikipedia.org/wiki/%D0%A2%D1%83%D1%...


In [14]:
# checking for correct parsing -> shouldn't be more 100 
final_wiki.groupby('page_1_name')['page_1_url'].count().sort_values() 

page_1_name
VI                             3
862 год                        7
Государственный терроризм      7
Жандарм Европы                23
Первомартовцы                 26
                            ... 
Иван Грозный                 100
Иван III Васильевич          100
Золотая орда                 100
Ленин, Владимир Ильич        100
Японская империя             100
Name: page_1_url, Length: 89, dtype: int64

In [15]:
# there are several cross-links in the dataset!
final_wiki.groupby('page_2_name')['page_2_url'].count().sort_values(ascending = False).head(10)

page_2_name
Российская империя       43
Повесть временных лет    29
Москва                   21
Николай II               20
1917 год                 20
Февральская революция    19
Санкт-Петербург          19
Николай I                19
Речь Посполитая          19
Русь                     19
Name: page_2_url, dtype: int64

In [16]:
# creating list with unique urls

all_url = [*final_wiki['page_1_url'], *final_wiki['page_2_url']]
print(len(all_url))
all_url = list(set(all_url))
len(all_url)

16300


4780

In [17]:
# create a distionary with ids
dict_url = dict(list(enumerate(all_url)))
keys = list(dict_url.keys())
values = list(dict_url.values())

In [18]:
# extract ids
ids_1 = []
for i in page_1_url: 
    for j in values:
        if i == j:
            ids_1.append(keys[values.index(j)]) 
    else:
        continue
        
ids_2 = []
for i in page_2_url: 
    for j in values:
        if i == j:
            ids_2.append(keys[values.index(j)]) 
    else:
        continue

In [19]:
# add columns to dataframe
final_wiki['id_1'] = ids_1
final_wiki['id_2'] = ids_2
ids_wiki = final_wiki[['id_1', 'id_2']]
ids_wiki.head(2)

,id_1,id_2
0,127,3336
1,127,3542


In [24]:
final_names_ids = final_wiki[['id_1', 'id_2', 'page_1_name', 'page_2_name']]

In [25]:
final_names_ids.to_csv('/Users/macbook/Desktop/ids_names.csv', index=False)

In [20]:
matrix = pd.crosstab(ids_wiki.id_1, ids_wiki.id_2)

In [21]:
idx = matrix.columns.union(matrix.index)

In [22]:
matrix1 = matrix.reindex(index = idx, columns=idx, fill_value=0)
matrix1

,0,1,2,3,4,5,6,7,8,9,...,4482,4483,4484,4485,4486,4487,4488,4489,4490,4491
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4487,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4488,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4489,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# check if matrix is correct
matrix1.loc[3828, [465, 436]] # interactions was detected

465    1
436    1
Name: 3828, dtype: int64

## Results

Please create a Jupyter Notebook file named as `lastname_hometask4.ipynb` (example: `terpilowski_hometask4.ipynb`) and place all the code in it. Create an archive with the generated files and Jupyter Notebook named as `lastname_hometask4.zip`.